In [1]:
import pandas as pd
import numpy as np

In [331]:
#df_company1 = pd.read_csv("companies.txt", sep="\t", encoding = "ISO-8859-1")
#df[col_list].applymap(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))

In [3]:
#import os
#os.getcwd()
#os.chdir('C:/Users/prateek.g/Desktop/UpGrad/Spark Group Project')

In [2]:
#Read the data set
df_company = pd.read_csv("companies.txt", sep="\t", encoding = "ISO-8859-1")
df_rounds2 = pd.read_csv("rounds2.csv")
df_mapping = pd.read_csv("mapping.csv")

In [3]:
#Table 1.1: Understand the Data Set

#How many unique companies are present in rounds2?
#Lower the company_permalink column of rounds 2 dataset to find the unique companies. 
df_rounds2_lower = df_rounds2['company_permalink'].str.lower()
df_rounds2_lower_uni = df_rounds2_lower.unique().tolist()
len(df_rounds2_lower_uni)

66368

In [4]:
#How many unique companies are present in the companies file?
#Lower the name column of company dataset to find the unique companies. 
df_company_lower = df_company['name'].str.lower()
df_company_lower_uni = df_company_lower.unique().tolist()
len(df_company_lower_uni)

66038

In [ ]:
#In the companies data frame, which column can be used as the unique key for each company? Write the name of the column.
#Permalink

In [5]:
#Are there any companies in the rounds2 file which are not  present in companies ? Answer Y/N.
#Split the company permalink column to find the names and compair with company dataset.
df_rounds2_split = df_rounds2_lower.str.split("/", n = 2, expand = True)
df_rounds2_split2 = df_rounds2_split[2] 
df_rounds2_split2_uni = df_rounds2_split2.unique()
len(df_rounds2_split2_uni)
#Answer = Y

66368

In [6]:
#Merge the two data frames so that all variables (columns) in the companies frame are added to the rounds2 data frame. 
#Name the merged frame master_frame. How many observations are present in master_frame ?

#merge the dataframe rounds2 and company as master_frame
df_rounds2["company_permalink"] = df_rounds2["company_permalink"].str.lower()
df_company["permalink"] = df_company["permalink"].str.lower()

master_frame = df_rounds2.merge(df_company, how='left', left_on='company_permalink', right_on='permalink')

In [7]:
#Table 2.1: Average Values of Investments for Each of these Funding Types 

#Average funding amount of venture type

#Filter the funding type to venture and replace Nan with 0 and calculate the mean of raised amount
df_rounds2_venture = df_rounds2[df_rounds2.funding_round_type == 'venture'] 
df_rounds2_venture = df_rounds2_venture.fillna(0)
df_rounds2_VentAvg = df_rounds2_venture["raised_amount_usd"].mean()/1000000
df_rounds2_VentAvg

10.634054436083181

In [8]:
#Average funding amount of angel type

#Filter the funding type to angel and replace Nan with 0 and calculate the mean of raised amount
df_rounds2_angel = df_rounds2[df_rounds2.funding_round_type == 'angel'] 
df_rounds2_angel = df_rounds2_angel.fillna(0)
df_rounds2_AngAvg = df_rounds2_angel["raised_amount_usd"].mean()/1000000
df_rounds2_AngAvg

0.7645643457499179

In [9]:
#Average funding amount of seed type

#Filter the funding type to seed and replace Nan with 0 and calculate the mean of raised amount
df_rounds2_seed = df_rounds2[df_rounds2.funding_round_type == 'seed'] 
df_rounds2_seed = df_rounds2_seed.fillna(0)
df_rounds2_SeedAvg = df_rounds2_seed["raised_amount_usd"].mean()/1000000
df_rounds2_SeedAvg

0.5566067416131568

In [10]:
#Average funding amount of private equity type

#Filter the funding type to private_equity and replace Nan with 0 and calculate the mean of raised amount
df_rounds2_prieqty = df_rounds2[df_rounds2.funding_round_type == 'private_equity'] 
df_rounds2_prieqty = df_rounds2_prieqty.fillna(0)
df_rounds2_PEAvg = df_rounds2_prieqty["raised_amount_usd"].mean()/1000000
df_rounds2_PEAvg

62.1117882297593

In [11]:
#Considering that Spark Funds wants to invest between 5 to 15 million USD per investment round, 
#which investment type is the most suitable for it?

#Creating a datafame to indentify investment type between 5 to 15 million
Fund_Type = pd.DataFrame(["venture","angel","seed","private_equity"])
Average = pd.DataFrame([df_rounds2_VentAvg,df_rounds2_AngAvg,df_rounds2_SeedAvg,df_rounds2_PEAvg]).round(2)
Average_Values_of_Investments = pd.concat([Fund_Type,Average],axis = 1)
Average_Values_of_Investments.columns = ["Investment_Type","Average_in_Millions"]
Average_Values_of_Investments

,Investment_Type,Average_in_Millions
0,venture,10.63
1,angel,0.76
2,seed,0.56
3,private_equity,62.11


In [12]:
#Checkpoint 3: Country Analysis
#For the chosen investment type, make a data frame named top9 with the top nine countries 
#(based on the total investment amount each country has received)

#As identified above venture is the suitable investment type create a df from master df filtering venture as investment type
master_frame_vent = master_frame[master_frame.funding_round_type == 'venture']

#To find the top countries, group the above df by country code and find the sum of raised amount
master_frame_top = master_frame_vent.groupby('country_code',as_index=False)['raised_amount_usd'].sum()

#Sort the df by raised amount to find top 9 countries
top = master_frame_top.sort_values(by = ['raised_amount_usd'],ascending=False)
top9 = top.head(9)
top9

,country_code,raised_amount_usd
100,USA,4.225108e+11
16,CHN,3.983542e+10
31,GBR,2.024563e+10
41,IND,1.439186e+10
13,CAN,9.583332e+09
30,FRA,7.259537e+09
44,ISR,6.907515e+09
23,DEU,6.346960e+09
47,JPN,3.363677e+09


In [13]:
#Identify the top three English-speaking countries in the data frame top9.

#Creating a new df to find top 3 english speaking countries from top 9 
Top_Countries = pd.DataFrame(["USA","GBR","IND"])
Top_Countries.columns=["country_code"]
Top_3_English_Speaking_Countries = pd.merge(top9,Top_Countries, left_on ="country_code", right_on ="country_code", how ='inner')
Top_3_English_Speaking_Countries

,country_code,raised_amount_usd
0,USA,4.225108e+11
1,GBR,2.024563e+10
2,IND,1.439186e+10


In [14]:
#Checkpoint 4: Sector Analysis 1

#Splitting the category list and find primary sector
category_list_split = master_frame['category_list'].str.split('|', n = 1,expand = True)
category_list_split.columns = ["primary_sector","category2"]
category_list_split1 = pd.DataFrame(category_list_split["primary_sector"])
category_list_split1

#Adding category_list_split to merge frame
merged_df = pd.concat([master_frame, category_list_split1], axis=1)

In [15]:
# replacing '0' with 'na' to clean category list
df_mapping.loc[df_mapping.category_list.notnull & df_mapping.category_list.str.contains('0'),'category_list']=df_mapping.loc[df_mapping.category_list.notnull & 
df_mapping.category_list.str.contains('0'),'category_list'].str.replace('0','na')

In [16]:
#Cleaning the mapping file

#Dropping the Blank columns in mapping file.
df_mapping1 = df_mapping.drop(columns=['Blanks'])
df_mapping2 = df_mapping1.drop([0,])

#To combine the true (1) values of main sector to category list 
New_mapping = pd.melt(df_mapping2, id_vars="category_list",)
New_mapping2 = New_mapping[New_mapping["value"] != 0]
New_mapping2 = New_mapping2.drop(['value'], axis = 1)

#Renaming the column names
New_mapping2.columns = ["category_list","Main_Sector"]

In [19]:
#Create a master_frame_final consisting of primary sectors mapped with main sector
master_frame_Final = pd.merge(merged_df,New_mapping2, left_on = "primary_sector", right_on = "category_list", how = "left")

In [20]:
#Cleaning the special characters in master df

#Cleaning the special characters from company_permalink
master_frame_Final['company_permalink'] = master_frame_Final[['company_permalink']].applymap(lambda x: str(x).encode('utf-8').decode('ascii', 'ignore'))

#Cleaning the special characters from permalink
master_frame_Final['permalink'] = master_frame_Final[['permalink']].applymap(lambda x: str(x).encode('utf-8').decode('ascii', 'ignore'))

#Cleaning the special characters from name
master_frame_Final['name'] = master_frame_Final[['name']].applymap(lambda x: str(x).encode('utf-8').decode('ascii', 'ignore'))

master_frame_Final.to_csv("Spark_Fund_Final_DF.csv")

In [21]:
#********************************************************Country1 USA**********************************************************
#Creating a new dataframe D1 for country 1 (USA) bassed on All the columns of the master_frame along with 
#the primary sector and the main sector falling within the 5-15 million USD range

#Filtering the master df with country code as USA, funding type as venture and raised amount in range of 5 to 15 million
D_1 = master_frame_Final[(master_frame_Final.country_code=="USA")& 
                        (master_frame_Final.funding_round_type=="venture")&
                        (master_frame_Final.raised_amount_usd>=5000000)&(master_frame_Final.raised_amount_usd<=15000000)]

#Grouping the df by Main_Sector to find the sum of raised amount and count of Main_Sector mapped with primary sector
D_1_Total_Count = D_1.groupby(['Main_Sector']).agg({'raised_amount_usd':'sum','Main_Sector':'count'})

#Renaming the new columns consisting of sum of raised amt and count of main sector
D_1_Total_Count_Rename = D_1_Total_Count.rename(columns={'Main_Sector':'Total_Count_Main_Sector','raised_amount_usd':'Total_Amount_Raised'})

#Creating merged dataframe D1 for USA
D1 = pd.merge(D_1,D_1_Total_Count_Rename,left_on='Main_Sector',right_index=True,how='inner')
D1.to_csv("D1_USA.csv")

In [649]:
#Table 5.1 : Sector-wise Investment Analysis

#5.1 Total number of investments (count)
Total_Number_Investment_1 = D_1_Total_Count_Rename.Total_Count_Main_Sector.sum()
Total_Number_Investment_1

12012

In [650]:
#5.2 Total amount of investment (USD)
Total_Amount_of_Investment_1 = D_1_Total_Count_Rename.Total_Amount_Raised.sum()
Total_Amount_of_Investment_1

107318294664.0

In [665]:
#5.3 Top sector (based on count of investments)
#5.4 Second-best sector (based on count of investments)
#5.5 Third-best sector (based on count of investments)
Top_Sector_BasedOn_Count_of_Investment = D_1_Total_Count_Rename.sort_values('Total_Count_Main_Sector',ascending=False)
Top_Sector_BasedOn_Count_of_Investment_1 = pd.DataFrame(Top_Sector_BasedOn_Count_of_Investment["Total_Count_Main_Sector"])
Top_Sector_BasedOn_Count_of_Investment_1.head(3)

,Total_Count_Main_Sector
Main_Sector,
Others,2950
"Social, Finance, Analytics, Advertising",2714
Cleantech / Semiconductors,2300


In [666]:
#5.6 Number of investments in the top sector
#5.7 Number of investments in the second-best sector
#5.8 Number of investments in the third-best sector
Top_Sector_Number_of_Investment = D_1_Total_Count_Rename.sort_values('Total_Amount_Raised',ascending=False)
Top_Sector_Number_of_Investment_1 = pd.DataFrame(Top_Sector_Number_of_Investment["Total_Amount_Raised"])
Top_Sector_Number_of_Investment_1.head(3)

,Total_Amount_Raised
Main_Sector,
Others,2.632101e+10
"Social, Finance, Analytics, Advertising",2.380738e+10
Cleantech / Semiconductors,2.120663e+10


In [667]:
#5.9 For the top sector count-wise (point 3), which company received the highest investment?
Top_Sector_CountWise_Company_1 = pd.DataFrame(D1[(D1.Main_Sector=="Others")].groupby('name')['raised_amount_usd'].sum().sort_values(ascending=False))
Top_Sector_CountWise_Company_1.head(1)

,raised_amount_usd
name,
Virtustream,64300000.0


In [658]:
#For the second-best sector count-wise (point 4), which company received the highest investment?
Second_Best_Sector_CountWise_Company_1 = pd.DataFrame(D1[(D1.Main_Sector=="Social, Finance, Analytics, Advertising")].groupby('name')['raised_amount_usd'].sum().sort_values(ascending=False))
Second_Best_Sector_CountWise_Company_1.head(1)

,raised_amount_usd
name,
SST Inc. (Formerly ShotSpotter),67933006.0


In [22]:
#********************************************************Country2 GBR**********************************************************
#Creating a new dataframe D2 for country 2 (GBR) bassed on All the columns of the master_frame along with 
#the primary sector and the main sector falling within the 5-15 million USD range

#Filtering the master df with country code as GBR, funding type as venture and raised amount in range of 5 to 15 million
D_2 = master_frame_Final[(master_frame_Final.country_code=="GBR")& 
                        (master_frame_Final.funding_round_type=="venture")&
                        (master_frame_Final.raised_amount_usd>=5000000)&(master_frame_Final.raised_amount_usd<=15000000)]

#Grouping the df by Main_Sector to find the sum of raised amount and count of Main_Sector mapped with primary sector
D_2_Total_Count = D_2.groupby(['Main_Sector']).agg({'raised_amount_usd':'sum','Main_Sector':'count'})

#Renaming the new columns consisting of sum of raised amt and count of main sector
D_2_Total_Count_Rename = D_2_Total_Count.rename(columns={'Main_Sector':'Total_Count_Main_Sector','raised_amount_usd':'Total_Amount_Raised'})

#Creating merged dataframe D1 for USA
D2 = pd.merge(D_2,D_2_Total_Count_Rename,left_on='Main_Sector',right_index=True,how='inner')
D2.to_csv("D2_USA.csv")

In [680]:
#Table 5.1 : Sector-wise Investment Analysis

#5.1 Total number of investments (count)
Total_Number_Investment_GBR_2 = D_2_Total_Count_Rename.Total_Count_Main_Sector.sum()
Total_Number_Investment_GBR_2

619

In [681]:
#5.2 Total amount of investment (USD)
Total_Amount_of_Investment_GBR_2 = D_2_Total_Count_Rename.Total_Amount_Raised.sum()
Total_Amount_of_Investment_GBR_2

5365228300.0

In [671]:
#5.3 Top sector (based on count of investments)
#5.4 Second-best sector (based on count of investments)
#5.5 Third-best sector (based on count of investments)
Top_Sector_BasedOn_Count_of_Investment_GBR = D_2_Total_Count_Rename.sort_values('Total_Count_Main_Sector',ascending=False)
Top_Sector_BasedOn_Count_of_Investment_GBR_2 = pd.DataFrame(Top_Sector_BasedOn_Count_of_Investment_GBR["Total_Count_Main_Sector"])
Top_Sector_BasedOn_Count_of_Investment_GBR_2.head(3)

,Total_Count_Main_Sector
Main_Sector,
Others,147
"Social, Finance, Analytics, Advertising",133
Cleantech / Semiconductors,128


In [673]:
#5.6 Number of investments in the top sector
#5.7 Number of investments in the second-best sector
#5.8 Number of investments in the third-best sector
Top_Sector_Number_of_Investment_GBR = D_2_Total_Count_Rename.sort_values('Total_Amount_Raised',ascending=False)
Top_Sector_Number_of_Investment_GBR_2 = pd.DataFrame(Top_Sector_Number_of_Investment["Total_Amount_Raised"])
Top_Sector_Number_of_Investment_GBR_2.head(3)

,Total_Amount_Raised
Main_Sector,
Others,2.632101e+10
"Social, Finance, Analytics, Advertising",2.380738e+10
Cleantech / Semiconductors,2.120663e+10


In [674]:
#5.9 For the top sector count-wise (point 3), which company received the highest investment?
Top_Sector_CountWise_Company_2 = pd.DataFrame(D2[(D2.Main_Sector=="Others")].groupby('name')['raised_amount_usd'].sum().sort_values(ascending=False))
Top_Sector_CountWise_Company_2.head(1)

,raised_amount_usd
name,
Electric Cloud,37000000.0


In [677]:
#For the second-best sector count-wise (point 4), which company received the highest investment?
Second_Best_Sector_CountWise_Company_2 = pd.DataFrame(D2[(D2.Main_Sector=="Social, Finance, Analytics, Advertising")].groupby('name')['raised_amount_usd'].sum().sort_values(ascending=False))
Second_Best_Sector_CountWise_Company_2.head(1)

,raised_amount_usd
name,
Celltick Technologies,37500000.0


In [23]:
#********************************************************Country3 IND**********************************************************
#Creating a new dataframe D1 for country 3 (IND) bassed on All the columns of the master_frame along with 
#the primary sector and the main sector falling within the 5-15 million USD range

#Filtering the master df with country code as IND, funding type as venture and raised amount in range of 5 to 15 million
D_3 = master_frame_Final[(master_frame_Final.country_code=="IND")& 
                        (master_frame_Final.funding_round_type=="venture")&
                        (master_frame_Final.raised_amount_usd>=5000000)&(master_frame_Final.raised_amount_usd<=15000000)]

#Grouping the df by Main_Sector to find the sum of raised amount and count of Main_Sector mapped with primary sector
D_3_Total_Count = D_3.groupby(['Main_Sector']).agg({'raised_amount_usd':'sum','Main_Sector':'count'})

#Renaming the new columns consisting of sum of raised amt and count of main sector
D_3_Total_Count_Rename = D_3_Total_Count.rename(columns={'Main_Sector':'Total_Count_Main_Sector','raised_amount_usd':'Total_Amount_Raised'})

#Creating merged dataframe D1 for USA
D3 = pd.merge(D_3,D_3_Total_Count_Rename,left_on='Main_Sector',right_index=True,how='inner')
D3.to_csv("D3_USA.csv")

In [683]:
#Table 5.1 : Sector-wise Investment Analysis

#5.1 Total number of investments (count)

Total_Number_Investment_IND_3 = D_3_Total_Count_Rename.Total_Count_Main_Sector.sum()
Total_Number_Investment_IND_3

328

In [685]:
#5.2 Total amount of investment (USD)
Total_Amount_of_Investment_IND_3 = D_3_Total_Count_Rename.Total_Amount_Raised.sum()
Total_Amount_of_Investment_IND_3

2949543602.0

In [686]:
#5.3 Top sector (based on count of investments)
#5.4 Second-best sector (based on count of investments)
#5.5 Third-best sector (based on count of investments)
Top_Sector_BasedOn_Count_of_Investment_IND = D_3_Total_Count_Rename.sort_values('Total_Count_Main_Sector',ascending=False)
Top_Sector_BasedOn_Count_of_Investment_IND_3 = pd.DataFrame(Top_Sector_BasedOn_Count_of_Investment_IND["Total_Count_Main_Sector"])
Top_Sector_BasedOn_Count_of_Investment_IND_3.head(3)

,Total_Count_Main_Sector
Main_Sector,
Others,110
"Social, Finance, Analytics, Advertising",60
"News, Search and Messaging",52


In [687]:
#5.6 Number of investments in the top sector
#5.7 Number of investments in the second-best sector
#5.8 Number of investments in the third-best sector
Top_Sector_Number_of_Investment_IND = D_3_Total_Count_Rename.sort_values('Total_Amount_Raised',ascending=False)
Top_Sector_Number_of_Investment_IND_3 = pd.DataFrame(Top_Sector_Number_of_Investment_IND["Total_Amount_Raised"])
Top_Sector_Number_of_Investment_IND_3.head(3)

,Total_Amount_Raised
Main_Sector,
Others,1.013410e+09
"Social, Finance, Analytics, Advertising",5.505496e+08
"News, Search and Messaging",4.338345e+08


In [688]:
#5.9 For the top sector count-wise (point 3), which company received the highest investment?
Top_Sector_CountWise_Company_3 = pd.DataFrame(D3[(D3.Main_Sector=="Others")].groupby('name')['raised_amount_usd'].sum().sort_values(ascending=False))
Top_Sector_CountWise_Company_3.head(1)

,raised_amount_usd
name,
FirstCry.com,39000000.0


In [689]:
#For the second-best sector count-wise (point 4), which company received the highest investment?
Second_Best_Sector_CountWise_Company_3 = pd.DataFrame(D3[(D3.Main_Sector=="Social, Finance, Analytics, Advertising")].groupby('name')['raised_amount_usd'].sum().sort_values(ascending=False))
Second_Best_Sector_CountWise_Company_3.head(1)

,raised_amount_usd
name,
Manthan Systems,50700000.0
